In [1]:
import math
import time
import tdmclient.notebook
import asyncio

await tdmclient.notebook.start()   





CancelledError: 

In [8]:
await tdmclient.notebook.stop()

In [10]:
# Get the first Thymio node
node = (await tdmclient.notebook.get_nodes())[0]


print("Connected to:", node._id if hasattr(node, "_id") else node)


Connected to: Node ff067687-6959-4893-b0ab-04a9b9fd9a6f


In [5]:
import sys, os

# Path to the project root (one level up from Core/)
project_root = os.path.abspath("..")

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Added to path:", project_root)


Added to path: c:\Users\danie\OneDrive\Documentos\FCUL\MobileRobots\ThymioBuilder


In [3]:
def wait_until_performed(var_name="on_performed", timeout=10):

    start = time.time()
    while time.time() - start < timeout:
        # Read variable from robot
        (value,) = get_var(var_name)

        if value is not None:

            if value == 1:
                print("✔ Action queue finished!")
                return True
            
            if value == -1:
                print("❌ Action queue encountered an error")
                return False

        time.sleep(0.1)  # avoid busy loop

    print("❌ Timeout waiting for action_queue to finish")
    return None

In [4]:
PRIMITIVE_MAP = {
    "F": [6],          # Forward
    "B": [1],          # Backwards full
    "HF": [2],         # Half forward
    "HB": [3],         # Half backwards
    "TL": [2, 5, 3],   # Turn left = half forward + rotate left + half back
    "TR": [2, 4, 3],   # Turn right = half forward + rotate right + half back
    "AB": [7, 8],          # Align Block +  Approach Block
}


def expand_actions(action_list):
    expanded = []
    for act in action_list:
        if act not in PRIMITIVE_MAP:
            print("Unknown action:", act)
            continue
        expanded.extend(PRIMITIVE_MAP[act])
    return expanded

EVENT_SIZE = 10

def make_payloads(action_list):
    expanded = expand_actions(action_list)
    payloads = []
    for i in range(0, len(expanded), EVENT_SIZE):
        chunk = expanded[i:i+EVENT_SIZE]
        chunk += [0] * (EVENT_SIZE - len(chunk))  # pad if shorter
        payloads.append(chunk)
    return payloads

async def send_action_queue_event(action_list):
    payloads = make_payloads(action_list)
    for idx, payload in enumerate(payloads):
      
        print(f"Sending payload {idx+1}/{len(payloads)}:", payload)
      
        await node.send_events({"action_queue": payload})
        # wait for robot confirmation before next payload
        wait_until_performed(var_name="on_performed", timeout=20)


In [2]:
def update_grid_for_planning(grid, block_manager, active_block_name, goal_pos, planner):
    """
    active_block_name: the block being transported RIGHT NOW
    goal_pos: (gx, gy) where it needs to be pushed
    """

    # Start with a clean grid
    grid.clear()

    for name, block in block_manager.blocks.items():

        # Skip the block being moved (so robot can navigate around/behind it)
        if name == active_block_name:
            continue

        # Mark all other blocks as obstacles
        grid.set_block(block)

    # Important: Mark the goal cell FREE
    gx, gy = goal_pos
    grid.set_cell(gx, gy, 0)

    planner.update_grid(grid)


In [3]:
def run_full_structure_build(grid, structure_plan, block_manager, planner, action_queue,
                             robot_start, robot_angle):
    """
    Build an entire structure of multiple blocks using your existing planner.
    """

    robot_pos = robot_start
    angle = robot_angle

    for block_name, target_pos in structure_plan.items():


        block = block_manager.get_block(block_name)

        update_grid_for_planning(grid, block_manager, block_name, target_pos, planner)


        if block is None:
            print(f"Block {block_name} not found!")
            continue

        block_start = (block.x, block.y)
        block_goal = target_pos

        print(f"\n=== BUILDING {block_name}: {block_start} -> {block_goal} ===")

        # 1. Plan mission
        cmds = planner.generate_mission(robot_pos, angle, block_start, block_goal)

        # Add to queue
        action_queue.add_sequence(cmds)

        # 2. Update block position logically
        block.x, block.y = block_goal

        robot_pos, angle = planner.get_robot_state()

        print(f"{block_name} placed. Robot now at {robot_pos}, facing {angle}")

    print("\n=== Structure Build Complete ===")


In [ ]:
import importlib
from Core.ActionQueue import ActionQueue
from Environment.Block_Manager import Block, BlockManager
from Environment.Grid_Map import GridMap
from PathPlanner import PathPlanner


block_manager = None
planner =   None
action_queue = None
grid = None

def set_up_envioronment():
   
   
    global block_manager, planner, action_queue, grid

   
    grid = GridMap(width_cells=6, height_cells=9, cell_size=1)


    block_manager = BlockManager()

    block_manager.add_block("cube1", Block(1, 1, 1, 1))
    block_manager.add_block("cube2", Block(3, 1, 1, 1)) 
    block_manager.add_block("cube3", Block(4, 3, 1, 1))
    block_manager.add_block("cube4", Block(1, 4, 1, 1))

    planner = PathPlanner(grid)
    action_queue = ActionQueue()

In [ ]:
set_up_envioronment()


global block_manager, planner, action_queue, grid

robot_start = (0, 0)
robot_angle = 90

run_full_structure_build(
    grid = grid,
    structure_plan = {
        "cube1": (1, 3),
        "cube2": (3, 2),
        "cube3": (3, 3),
        "cube4": (2, 3),
    },
    block_manager=block_manager,
    planner=planner,
    action_queue=action_queue,
    robot_start=robot_start,
    robot_angle=robot_angle
)

action_queue.print_queue()
#await send_action_queue_event(action_queue.actions)



=== BUILDING cube1: (1, 1) -> (1, 4) ===
--- PLANNING MISSION ---
Robot: (0, 0) facing 0
Block: (1, 1) -> (1, 4)
Block Path: [(1, 1), (1, 2), (1, 3), (1, 4)]
Phase 1 (Approach): 2 moves
Phase 2 (Transport): 4 moves
cube1 placed. Robot now at (1, 3), facing 90

=== BUILDING cube2: (3, 3) -> (4, 4) ===
--- PLANNING MISSION ---
Robot: (1, 3) facing 90
Block: (3, 3) -> (4, 4)
Block Path: [(3, 3), (4, 3), (4, 4)]
Phase 1 (Approach): 2 moves
Phase 2 (Transport): 9 moves
cube2 placed. Robot now at (4, 3), facing 90

=== Structure Build Complete ===
F -> TR -> AB -> F -> F -> F -> TL -> F -> AB -> F -> TL -> F -> TR -> F -> TR -> AB -> F
